In [10]:
import os
import joblib
import numpy as np
import pandas as pd
import polars as pl

import pandas.api.types
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import lifelines
from lifelines.utils import concordance_index

import sys

print(sys.version)
for i in [np, pd, pl, mpl, sns, lifelines, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        print(i.__name__)

3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]
numpy 1.26.4
pandas 2.2.3
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
lifelines 0.30.0
sklearn 1.5.2
lightgbm
xgboost 2.1.2
catboost 1.2.5


In [11]:
import dproc, sgutil, sgpp, sgml, custpp
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit, train_test_split
from sklearn.impute import SimpleImputer
from lifelines import NelsonAalenFitter

In [12]:
data_path = 'data'
model_path = 'model'

p2 =joblib.load(os.path.join(model_path, 'p2.joblib'))
df_train = p2.transform([os.path.join(data_path, 'train.csv')])

In [25]:
import sgml
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

def predict_func(m, df, X):
    return pd.Series(m.predict(df[X]), index = df.index)

def score(df, prds):
    return df.groupby('race_group', observed=True).apply(
        lambda x: concordance_index(x['efs_time'], -prds.loc[x.index], x['efs']), include_groups=False
    ).pipe(
        lambda x: float(x.mean() - x.std(ddof=0))
    )

lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor)
cb_adapter = sgml.CBAdapter(cb.CatBoostRegressor)

In [18]:
def get_kmf_predictor():
    kmf = [
        ('lgb1', lgb_adapter), ('xgb1', xgb_adapter), ('cb1', cb_adapter)
    ]
    kmf_prd = {
        n: sgml.make_predictor(model_path, n, a, predict_func) for n, a in kmf
    }
    return lambda x: pd.concat([
        p(x).rename(n) for n, p in kmf_prd.items()
    ], axis=1)
kmf_predictor = get_kmf_predictor()

In [23]:
def get_naf_predictor():
    kmf = [
        ('lgb2', lgb_adapter), ('xgb2', xgb_adapter), ('cb2', cb_adapter)
    ]
    kmf_prd = {
        n: sgml.make_predictor(model_path, n, a, predict_func) for n, a in kmf
    }
    return lambda x: pd.concat([
        p(x).rename(n) for n, p in kmf_prd.items()
    ], axis=1)
naf_predictor = get_naf_predictor()

In [21]:
kmf_train_prd = kmf_predictor(df_train).dot([0.4, 0.3, 0.3]).rename('kmf')

In [24]:
naf_train_prd = naf_predictor(df_train).dot([0.4, 0.3, 0.3]).rename('naf')

In [33]:
from scipy.stats import rankdata
score(
    df_train,
    pd.concat([
        kmf_train_prd.rank(),
        naf_train_prd.rank()
    ], axis=1).mean(axis=1)
)

0.732103843905857